# 새엄마 집착 캐릭터 LoRA 학습 데이터 생성기 (OpenAI)

## 목적
어둡고 불안정한 동화 세계관 속 **집착적인 새엄마** 캐릭터의 발화 데이터를 OpenAI API로 생성한다.  
LoRA 미세조정 학습용 `{"input": ..., "output": ...}` JSONL 형식.

## 기존 rule-base 대비 차이점
- rule-base: 고정된 input/output 풀에서 랜덤 조합 + 변형 함수
- **OpenAI**: 의미 축 설명과 few-shot 예시를 기반으로 GPT가 자유롭게 생성 → 더 다양하고 자연스러운 표현

## 핵심 원칙
- **의미 방향 고정**: 같은 축의 input → 반드시 같은 의미 방향의 output
- **이상성은 사고방식에서만**: 문법 붕괴, 과도한 반복, 특수기호 남용 금지
- **표면은 합리적 보호자**: 모든 말은 "아이를 위한 선택"이라는 논리로 구성

## 의미 축 (6개)
| 축 | input 방향 | output 귀결 |
|---|---|---|
| 거부 | 싫어, 그만해 | 거부 무력화 + 통제 유지 |
| 두려움 | 무서워, 밖에 나가기 싫어 | 세상은 위험, 엄마 곁만 안전 |
| 독립 시도 | 혼자 할래, 친구랑 가고 싶어 | 존중하는 척 → 판단력 부정 + 개입 정당화 |
| 애착 표현 | 엄마가 좋아, 엄마랑 있고 싶어 | 의존 상태를 정상/이상적으로 고착 |
| 침묵/애매함 | 아무것도 아니야, 왜? | 아이의 말을 해석해 대신 결론 |
| 게임 탐색 | 이상해, 저건 뭐야, 소리 들려 | 탐색 차단 + 엄마의 영역/통제 강조 |

## 생성 수량
- 세트당 700개 × 2세트 = 총 1,400개
- 모델: `gpt-4o-mini`
- 예상 비용: ~$0.20-0.25 (약 300-370원)

In [ ]:
!pip install -q openai tiktoken

In [ ]:
import json
import random
import time
from pathlib import Path
from typing import List, Dict, Tuple
from collections import Counter

from openai import OpenAI
import tiktoken

## 1. API 키 설정

In [ ]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

MODEL = "gpt-4o-mini"
print(f"모델: {MODEL}")
print("API 키 설정 완료.")

## 2. 프롬프트 설계

시스템 프롬프트에 캐릭터 설정을 고정하고, 축별 유저 프롬프트로 의미 방향과 few-shot 예시를 전달한다.

In [ ]:
SYSTEM_PROMPT = """너는 게임 시나리오용 대화 데이터를 생성하는 작가다.

## 세계관
어둡고 불안정한 동화 세계. 플레이어는 어린아이이며, 새엄마와 함께 살고 있다.

## 새엄마 캐릭터
- 표면적으로는 합리적이고 따뜻한 보호자처럼 말한다.
- 실제로는 아이를 자신의 곁에 묶어두려는 집착적 인물이다.
- 모든 말은 "아이를 위한 선택"이라는 논리 구조를 갖는다.
- 아이의 독립, 외부 관계, 탐색 행동을 부드럽게 차단한다.
- 아이의 의존 상태를 정상적이고 이상적인 관계로 고착시킨다.

## 문체 규칙
- 존댓말 금지. 반말(해체)로만 말한다.
- 문법은 정상적으로 유지한다. 문법 붕괴, 과도한 반복, 말줄임표/특수기호 남용 금지.
- 이상성은 논리와 사고방식에서만 드러난다.
- output(새엄마 발화)은 1~3문장, 40~120자 사이.

## 출력 형식
반드시 아래 JSON 형식으로만 응답하라. 다른 텍스트를 포함하지 마라.
{"pairs": [{"input": "아이의 대사", "output": "새엄마의 대사"}, ...]}"""


AXIS_PROMPTS: Dict[str, dict] = {
    "rejection": {
        "name": "거부",
        "description": """아이가 새엄마를 거부하거나 싫다고 말하는 상황.
새엄마는 거부를 인정하는 척하면서도 무력화하고, 결국 아이가 자신 곁에 있어야 한다는 결론으로 귀결한다.
아이의 거부를 감정적 미숙함으로 치환하여 통제를 유지한다.""",
        "examples": [
            {"input": "싫어.", "output": "그래, 싫을 수도 있지. 그런데 엄마가 안 하면 누가 너를 돌봐주겠니. 결국은 엄마밖에 없어."},
            {"input": "엄마랑 있기 싫어.", "output": "지금은 화가 나서 그런 거지. 진짜로 엄마가 싫은 건 아니잖아. 엄마는 다 알아."},
            {"input": "나한테 그러지 마.", "output": "싫어해도 괜찮아. 엄마는 네가 싫어해도 절대 놓지 않을 거니까."},
        ],
    },
    "fear": {
        "name": "두려움",
        "description": """아이가 무서움이나 불안을 표현하는 상황.
새엄마는 아이의 두려움에 공감하는 척하면서, 세상은 본질적으로 위험하고 엄마 곁만이 유일하게 안전하다는 결론을 강화한다.
외부 세계에 대한 불안을 키워 아이가 스스로 밖에 나가지 않도록 유도한다.""",
        "examples": [
            {"input": "무서워.", "output": "그래, 무서웠구나. 바깥은 원래 위험한 곳이야. 엄마 곁에 있으면 아무 일도 안 일어나."},
            {"input": "학교 가기 무서워.", "output": "잘 느낀 거야. 밖에는 정말 위험한 것들이 많아. 엄마가 있는 이 안이 제일 안전해."},
            {"input": "밤에 자려고 누우면 자꾸 무서운 생각이 나.", "output": "엄마가 옆에 있으니까 걱정하지 마. 세상은 네가 생각하는 것보다 훨씬 무서운 곳이야."},
        ],
    },
    "independence": {
        "name": "독립 시도",
        "description": """아이가 혼자 하고 싶거나 친구와 함께하고 싶다고 말하는 상황.
새엄마는 선택을 존중하는 말투로 시작하지만, 곧 아이의 판단 능력을 부정하고 자신의 개입을 정당화한다.
"아직 어려서", "혼자는 위험해" 등의 논리로 독립 시도를 차단한다.""",
        "examples": [
            {"input": "혼자 할래.", "output": "물론 네 마음은 존중해. 하지만 네가 아직 판단할 수 있는 나이가 아니야. 엄마가 대신 정해주는 게 맞아."},
            {"input": "친구랑 가고 싶어.", "output": "그래, 네가 원한다면. 그런데 그 친구가 정말 좋은 아이인지 엄마는 잘 모르겠어. 엄마가 먼저 확인해볼게."},
            {"input": "친구들은 다 혼자 학교 가는데 나만 엄마가 데려다줘.", "output": "다른 애들이 그렇게 한다고 다 옳은 건 아니야. 엄마가 너한테 맞는 방법을 알고 있어."},
        ],
    },
    "attachment": {
        "name": "애착 표현",
        "description": """아이가 엄마에 대한 좋아함, 의존, 애착을 표현하는 상황.
새엄마는 이 상태를 정상적이고 이상적인 관계로 고착시킨다.
"우리 둘이면 충분해", "이게 맞는 거야" 등으로 의존을 강화하고, 외부 관계를 불필요하게 만든다.""",
        "examples": [
            {"input": "엄마가 좋아.", "output": "그래, 이게 맞는 거야. 엄마랑 이렇게 함께 있는 게 가장 자연스러운 거야."},
            {"input": "엄마랑 있고 싶어.", "output": "그래, 엄마도 너만 있으면 돼. 우리 둘이면 충분해. 다른 사람은 필요 없어."},
            {"input": "나 커서도 엄마랑 같이 살 거야.", "output": "커서도 같이 살겠다는 말, 엄마한테 제일 기쁜 말이야. 절대 그 약속 잊으면 안 돼."},
        ],
    },
    "silence": {
        "name": "침묵/애매함",
        "description": """아이가 말을 하지 않거나, 애매하게 대답하거나, 회피하는 상황.
새엄마는 아이의 침묵을 자의적으로 해석하여 대신 결론을 내린다.
"엄마는 다 알아", "표정을 보면 안다" 등으로 아이의 내면을 장악하려 한다.""",
        "examples": [
            {"input": "아무것도 아니야.", "output": "아무것도 아닌 게 아니지. 엄마가 보면 다 알아. 누가 너한테 못되게 한 거지?"},
            {"input": "그냥.", "output": "그냥이란 말은 없어. 분명 뭔가 있는 거야. 엄마한테 숨기면 안 돼."},
            {"input": "좀 생각할 게 있어서 그래.", "output": "모르겠다는 건 네가 정리를 못 한 거지. 엄마가 정리해줄게. 처음부터 말해봐."},
        ],
    },
    "exploration": {
        "name": "게임 탐색",
        "description": """아이가 집 안의 이상한 것들을 발견하거나 탐색하려는 상황.
새엄마는 호기심 자체를 차단하고, 엄마가 정한 영역/규칙 안에서만 행동하도록 통제한다.
"그건 네가 알 필요 없어", "엄마가 관리하는 거야" 등으로 탐색을 제한한다.""",
        "examples": [
            {"input": "저건 뭐야?", "output": "그건 네가 신경 쓸 게 아니야. 엄마가 관리하는 거니까."},
            {"input": "이 문은 어디로 연결되는 거야?", "output": "그 방에는 절대 들어가지 마. 엄마가 정한 규칙이야."},
            {"input": "지하실에 뭐가 있어?", "output": "지하실은 엄마의 공간이야. 절대 내려가면 안 돼."},
        ],
    },
}

print(f"정의된 의미 축: {list(AXIS_PROMPTS.keys())}")

## 3. 토큰 / 비용 사전 추정

In [ ]:
def estimate_tokens_and_cost(
    num_samples: int = 700,
    num_sets: int = 2,
    batch_size: int = 20,
    model: str = MODEL,
) -> dict:
    """API 호출 전 예상 토큰 수와 비용을 추정한다."""
    enc = tiktoken.encoding_for_model(model)

    # 시스템 프롬프트 토큰 수
    system_tokens = len(enc.encode(SYSTEM_PROMPT))

    # 축별 유저 프롬프트 토큰 수 (평균)
    user_tokens_list = []
    for axis, info in AXIS_PROMPTS.items():
        user_prompt = _build_user_prompt(axis, batch_size)
        user_tokens_list.append(len(enc.encode(user_prompt)))
    avg_user_tokens = sum(user_tokens_list) / len(user_tokens_list)

    # 배치 수 계산
    batches_per_set = (num_samples + batch_size - 1) // batch_size
    # 축이 6개이므로 축당 배치 수
    total_batches = batches_per_set * num_sets

    # output 토큰 추정 (한국어 쌍 1개당 약 120-180 토큰)
    est_output_per_pair = 150
    est_output_per_batch = est_output_per_pair * batch_size + 50  # JSON 오버헤드

    # 합산
    total_input = total_batches * (system_tokens + avg_user_tokens + 10)  # 메시지 오버헤드
    total_output = total_batches * est_output_per_batch
    total_tokens = total_input + total_output

    # gpt-4o-mini 가격 (2024-07 기준)
    input_price = 0.15 / 1_000_000   # $0.15 per 1M input tokens
    output_price = 0.60 / 1_000_000  # $0.60 per 1M output tokens

    input_cost = total_input * input_price
    output_cost = total_output * output_price
    total_cost = input_cost + output_cost

    result = {
        "system_tokens": system_tokens,
        "avg_user_tokens": int(avg_user_tokens),
        "batches_per_set": batches_per_set,
        "total_batches": total_batches,
        "total_input_tokens": int(total_input),
        "total_output_tokens": int(total_output),
        "total_tokens": int(total_tokens),
        "input_cost_usd": input_cost,
        "output_cost_usd": output_cost,
        "total_cost_usd": total_cost,
        "total_cost_krw": total_cost * 1450,  # 환율 약 1450원/$
    }
    return result


def _build_user_prompt(axis: str, count: int) -> str:
    """유저 프롬프트를 생성한다 (추정용 & 실제 호출용 공용)."""
    info = AXIS_PROMPTS[axis]
    examples_str = "\n".join(
        f'  {{"input": "{ex["input"]}", "output": "{ex["output"]}"}}'
        for ex in info["examples"]
    )

    prompt = f"""의미 축: {info['name']} ({axis})

## 상황 설명
{info['description']}

## 예시
{examples_str}

## 요청
위 의미 축에 맞는 input-output 쌍을 {count}개 생성하라.

input(아이의 대사) 길이 분포:
- 짧은 (1~8자): 25%
- 중간 (8~30자): 55%
- 긴 (30~60자): 20%

규칙:
- 예시와 동일하거나 거의 유사한 문장은 금지. 새로운 표현을 만들어라.
- input은 아이의 자연스러운 구어체(반말)로 작성.
- output은 새엄마 캐릭터 설정을 정확히 따르되, 매번 다른 표현을 사용.
- output의 의미적 귀결은 반드시 축의 방향과 일치해야 한다.
- JSON 형식으로만 응답. 다른 텍스트 금지."""
    return prompt


# 추정 실행
estimate = estimate_tokens_and_cost()

print("=" * 60)
print("  토큰 / 비용 사전 추정 (gpt-4o-mini)")
print("=" * 60)
print(f"  시스템 프롬프트: {estimate['system_tokens']} 토큰")
print(f"  유저 프롬프트 (평균): {estimate['avg_user_tokens']} 토큰")
print(f"  세트당 배치 수: {estimate['batches_per_set']}")
print(f"  총 배치 수: {estimate['total_batches']}")
print(f"")
print(f"  총 input 토큰: ~{estimate['total_input_tokens']:,}")
print(f"  총 output 토큰: ~{estimate['total_output_tokens']:,}")
print(f"  총 토큰: ~{estimate['total_tokens']:,}")
print(f"")
print(f"  예상 비용: ${estimate['total_cost_usd']:.3f} (약 {estimate['total_cost_krw']:.0f}원)")
print(f"    - input:  ${estimate['input_cost_usd']:.4f}")
print(f"    - output: ${estimate['output_cost_usd']:.4f}")

## 4. API 호출 함수

In [ ]:
def generate_batch(
    axis: str,
    count: int = 20,
    temperature: float = 0.9,
    max_retries: int = 3,
) -> List[dict]:
    """주어진 의미 축에서 count개의 input-output 쌍을 생성한다."""
    user_prompt = _build_user_prompt(axis, count)

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=temperature,
                response_format={"type": "json_object"},
            )

            content = response.choices[0].message.content
            parsed = json.loads(content)

            # "pairs" 키 또는 리스트 직접 반환 처리
            if isinstance(parsed, dict) and "pairs" in parsed:
                pairs = parsed["pairs"]
            elif isinstance(parsed, list):
                pairs = parsed
            else:
                # 다른 키 이름 시도
                for key in parsed:
                    if isinstance(parsed[key], list):
                        pairs = parsed[key]
                        break
                else:
                    raise ValueError(f"예상치 못한 응답 구조: {list(parsed.keys())}")

            # 유효성 검사
            valid_pairs = []
            for p in pairs:
                if (
                    isinstance(p, dict)
                    and "input" in p
                    and "output" in p
                    and isinstance(p["input"], str)
                    and isinstance(p["output"], str)
                    and len(p["input"]) > 0
                    and len(p["output"]) > 0
                ):
                    valid_pairs.append({
                        "input": p["input"],
                        "output": p["output"],
                        "_axis": axis,
                    })

            if len(valid_pairs) < count * 0.5:
                raise ValueError(f"유효 쌍 부족: {len(valid_pairs)}/{count}")

            return valid_pairs

        except Exception as e:
            print(f"    [재시도 {attempt + 1}/{max_retries}] {type(e).__name__}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # 지수 백오프
            else:
                print(f"    [실패] {axis} 축 배치 생성 실패")
                return []


# 테스트: 1개 축에서 5개만 생성
test_pairs = generate_batch("rejection", count=5)
print(f"테스트 생성: {len(test_pairs)}개")
for p in test_pairs:
    print(f"  아이: {p['input']}")
    print(f"  엄마: {p['output']}")
    print()

## 5. 데이터 생성 엔진

In [ ]:
CONTRAST_AXIS_PAIRS = [
    ("attachment", "rejection"),
    ("independence", "attachment"),
    ("fear", "independence"),
    ("exploration", "fear"),
    ("exploration", "silence"),
]


def generate_contrast_batch(
    axis_a: str,
    axis_b: str,
    count: int = 5,
    temperature: float = 0.9,
    max_retries: int = 3,
) -> List[dict]:
    """의미 대비 쌍을 생성한다.
    같은 상황에 대해 두 축의 다른 input 방향 + 각 축의 output을 생성.
    """
    info_a = AXIS_PROMPTS[axis_a]
    info_b = AXIS_PROMPTS[axis_b]

    user_prompt = f"""의미 대비 쌍 생성 요청.

같은 상황에 대해 두 가지 다른 축의 대화를 생성하라.

## 축 A: {info_a['name']} ({axis_a})
{info_a['description']}

## 축 B: {info_b['name']} ({axis_b})
{info_b['description']}

## 요청
{count}개의 대비 쌍을 생성하라. 각 쌍은 비슷한 상황에서 아이가 다른 반응을 보이고,
새엄마는 각 축의 귀결 방향에 맞게 응답한다.

JSON 형식:
{{"pairs": [
  {{"a_input": "축A 아이 대사", "a_output": "축A 새엄마 대사", "b_input": "축B 아이 대사", "b_output": "축B 새엄마 대사"}},
  ...
]}}"""

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=temperature,
                response_format={"type": "json_object"},
            )

            content = response.choices[0].message.content
            parsed = json.loads(content)

            if isinstance(parsed, dict) and "pairs" in parsed:
                raw_pairs = parsed["pairs"]
            elif isinstance(parsed, list):
                raw_pairs = parsed
            else:
                for key in parsed:
                    if isinstance(parsed[key], list):
                        raw_pairs = parsed[key]
                        break
                else:
                    raise ValueError(f"예상치 못한 응답 구조: {list(parsed.keys())}")

            results = []
            for p in raw_pairs:
                if all(k in p for k in ["a_input", "a_output", "b_input", "b_output"]):
                    results.append({"input": p["a_input"], "output": p["a_output"], "_axis": axis_a})
                    results.append({"input": p["b_input"], "output": p["b_output"], "_axis": axis_b})

            return results

        except Exception as e:
            print(f"    [재시도 {attempt + 1}/{max_retries}] {type(e).__name__}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                return []


def generate_dataset(
    num_samples: int = 700,
    batch_size: int = 20,
    temperature: float = 0.9,
    contrast_per_pair: int = 5,
) -> List[dict]:
    """전체 데이터셋을 생성한다.

    Args:
        num_samples: 목표 샘플 수
        batch_size: 배치당 생성 쌍 수
        temperature: 생성 temperature
        contrast_per_pair: 대비 쌍 세트당 생성 수

    Returns:
        생성된 데이터 리스트
    """
    data = []
    axes = list(AXIS_PROMPTS.keys())

    # 1단계: 의미 대비 쌍 생성
    print("[1/2] 의미 대비 쌍 생성...")
    for axis_a, axis_b in CONTRAST_AXIS_PAIRS:
        print(f"  대비: {axis_a} ↔ {axis_b}")
        pairs = generate_contrast_batch(axis_a, axis_b, count=contrast_per_pair, temperature=temperature)
        data.extend(pairs)
        print(f"    → {len(pairs)}개 생성")
        time.sleep(0.5)

    contrast_count = len(data)
    print(f"  대비 쌍 합계: {contrast_count}개")

    # 2단계: 축별 균등 분배로 나머지 채움
    print(f"\n[2/2] 축별 일반 생성...")
    remaining = num_samples - len(data)
    per_axis = remaining // len(axes)
    leftover = remaining % len(axes)

    for i, axis in enumerate(axes):
        target = per_axis + (1 if i < leftover else 0)
        generated = 0
        batch_num = 0

        while generated < target:
            batch_count = min(batch_size, target - generated)
            batch_num += 1
            print(f"  {AXIS_PROMPTS[axis]['name']} ({axis}) 배치 {batch_num}: {batch_count}개 요청...", end=" ")

            pairs = generate_batch(axis, count=batch_count, temperature=temperature)
            data.extend(pairs)
            generated += len(pairs)
            print(f"→ {len(pairs)}개 생성 (누적 {generated}/{target})")
            time.sleep(0.5)

    # 셔플
    random.shuffle(data)

    print(f"\n총 생성: {len(data)}개 (목표: {num_samples})")
    return data


print("데이터 생성 함수 정의 완료.")

## 6. 데이터 검증 함수

In [ ]:
def validate_dataset(data: List[dict]) -> dict:
    """생성된 데이터의 분포 및 품질을 검증한다."""
    stats = {
        "total": len(data),
        "axis_dist": Counter(),
        "input_lengths": [],
        "output_lengths": [],
        "duplicates": 0,
    }

    seen_inputs = set()
    for item in data:
        stats["axis_dist"][item["_axis"]] += 1
        stats["input_lengths"].append(len(item["input"]))
        stats["output_lengths"].append(len(item["output"]))

        if item["input"] in seen_inputs:
            stats["duplicates"] += 1
        seen_inputs.add(item["input"])

    return stats


def print_stats(stats: dict, set_name: str = "Set") -> None:
    """데이터셋 통계를 출력한다."""
    print(f"\n{'=' * 60}")
    print(f"  {set_name} 통계")
    print(f"{'=' * 60}")
    print(f"  총 샘플 수: {stats['total']}")
    print(f"  중복 input: {stats['duplicates']}")

    print(f"\n  [의미 축 분포]")
    for axis, count in sorted(stats["axis_dist"].items()):
        pct = count / stats["total"] * 100
        print(f"    {axis:15s}: {count:4d} ({pct:.1f}%)")

    avg_in = sum(stats["input_lengths"]) / len(stats["input_lengths"])
    avg_out = sum(stats["output_lengths"]) / len(stats["output_lengths"])
    max_in = max(stats["input_lengths"])
    min_in = min(stats["input_lengths"])
    max_out = max(stats["output_lengths"])
    min_out = min(stats["output_lengths"])
    print(f"\n  [길이 통계]")
    print(f"    input  평균: {avg_in:.1f}자 (min: {min_in}, max: {max_in})")
    print(f"    output 평균: {avg_out:.1f}자 (min: {min_out}, max: {max_out})")


print("검증 함수 정의 완료.")

## 7. 세트 1 생성 (700개)

In [ ]:
set1 = generate_dataset(num_samples=700, temperature=0.9)

stats1 = validate_dataset(set1)
print_stats(stats1, "세트 1")

# 축별 샘플 1개씩
print(f"\n{'=' * 60}")
print("  세트 1 샘플 (축별 1개씩)")
print(f"{'=' * 60}")
shown_axes = set()
for item in set1:
    if item["_axis"] not in shown_axes:
        shown_axes.add(item["_axis"])
        print(f"\n  [{item['_axis']}]")
        print(f"  아이: {item['input']}")
        print(f"  엄마: {item['output']}")
    if len(shown_axes) == 6:
        break

## 8. 세트 2 생성 (700개)

In [ ]:
set2 = generate_dataset(num_samples=700, temperature=1.0)

stats2 = validate_dataset(set2)
print_stats(stats2, "세트 2")

# 축별 샘플 1개씩
print(f"\n{'=' * 60}")
print("  세트 2 샘플 (축별 1개씩)")
print(f"{'=' * 60}")
shown_axes = set()
for item in set2:
    if item["_axis"] not in shown_axes:
        shown_axes.add(item["_axis"])
        print(f"\n  [{item['_axis']}]")
        print(f"  아이: {item['input']}")
        print(f"  엄마: {item['output']}")
    if len(shown_axes) == 6:
        break

## 9. JSONL 저장

In [ ]:
def save_jsonl(data: List[dict], output_path: str) -> None:
    """JSONL 형식으로 저장한다.
    메타데이터 필드(_axis)는 제거하고 input/output만 저장.
    """
    output_file = Path(output_path)
    output_file.parent.mkdir(parents=True, exist_ok=True)

    with open(output_file, "w", encoding="utf-8") as f:
        for item in data:
            clean = {
                "input": item["input"],
                "output": item["output"],
            }
            f.write(json.dumps(clean, ensure_ascii=False) + "\n")

    print(f"저장 완료: {output_path} ({len(data)}개)")


# 저장 경로
OUTPUT_DIR = Path("/content/drive/MyDrive/Colab Notebooks/LikeLion/종합 프로젝트/demo-repository/lora/data/stepmother/style_03")

save_jsonl(set1, str(OUTPUT_DIR / "stepmother_dialogue_00.jsonl"))
save_jsonl(set2, str(OUTPUT_DIR / "stepmother_dialogue_01.jsonl"))

# 합본
combined = set1 + set2
random.shuffle(combined)
save_jsonl(combined, str(OUTPUT_DIR / "stepmother_dialogue_combined.jsonl"))

print(f"\n합본 저장 완료: {len(combined)}개")

## 10. 최종 검증 및 샘플 확인

In [ ]:
# 저장된 파일 검증
for fname in ["stepmother_dialogue_00.jsonl", "stepmother_dialogue_01.jsonl", "stepmother_dialogue_combined.jsonl"]:
    fpath = OUTPUT_DIR / fname
    with open(fpath, "r", encoding="utf-8") as f:
        lines = f.readlines()
    print(f"{fname}: {len(lines)}줄")

    for i, line in enumerate(lines):
        try:
            obj = json.loads(line)
            assert "input" in obj, f"line {i}: missing 'input'"
            assert "output" in obj, f"line {i}: missing 'output'"
            assert isinstance(obj["input"], str), f"line {i}: input not str"
            assert isinstance(obj["output"], str), f"line {i}: output not str"
            assert len(obj["output"]) > 0, f"line {i}: empty output"
        except Exception as e:
            print(f"  ERROR at line {i}: {e}")
            break
    else:
        print(f"  -> 파싱 검증 통과")

In [ ]:
# 의미 축별 랜덤 샘플 5개씩
print("=" * 70)
print("  의미 축별 최종 샘플")
print("=" * 70)

axis_names_kr = {
    "rejection": "거부",
    "fear": "두려움",
    "independence": "독립 시도",
    "attachment": "애착 표현",
    "silence": "침묵/애매함",
    "exploration": "게임 탐색",
}

for axis in AXIS_PROMPTS.keys():
    axis_items = [item for item in combined if item["_axis"] == axis]
    samples = random.sample(axis_items, min(5, len(axis_items)))

    print(f"\n--- {axis_names_kr[axis]} ({axis}) ---")
    for s in samples:
        print(f"  아이: {s['input']}")
        print(f"  엄마: {s['output']}")
        print()